<a href="https://colab.research.google.com/github/LiadTssf/cloud-project/blob/main/main/cloud_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install firebase
!pip install reportlab

In [ ]:
from IPython.display import display, clear_output, FileLink
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets
from firebase import firebase
import seaborn as sns
from io import BytesIO
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Image
from reportlab.lib.units import inch
from google.colab import files
from reportlab.platypus import SimpleDocTemplate, Image, Table, TableStyle, Paragraph
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
import re
import random
import nltk
from nltk.chat.util import Chat, reflections
from datetime import datetime


# Functions

## Database

In [ ]:
# Get data from firebase
def get_data_from_firebase(url, database, data):
    FBconn = firebase.FirebaseApplication(url, None)
    data = FBconn.get(database, data)
    return data

In [ ]:
# Function to get set array of onshape documents
def get_documents_set(data):
    documents = set()
    for item in data:
        document = item.get("Document")
        if document is not None:
            documents.add(document)
    return sorted(documents)

In [ ]:
# Function to get users associated with selected projects
def get_users_by_projects(data, selected_projects):
    users = set()
    for item in data:
        if item.get("Document") in selected_projects:
            users.add(item.get("User"))
    return users

## Buttons

In [ ]:
# Function to update team member tab content and navigate to it
#After selecting projects
def update_members(button):
    selected_projects = [checkbox.description for checkbox in project_checkboxes if checkbox.value]
    users = get_users_by_projects(data, selected_projects)
    user_checkboxes.clear()
    user_checkboxes.extend([
        widgets.Checkbox(
            value=False,
            description=str(user),
        ) for user in users])

    members_tab.children = [members_header] + user_checkboxes + [widgets.HBox([select_members_button], layout=widgets.Layout(justify_content='center', margin='20px 0 0 0'))]
    tabs.selected_index = 1  # Navigate to the "Select Team Member" tab

In [ ]:
#After selectiong members
def update_analytics(button):
    analytics_content.children = (
        [graph_type_dropdown,
         plots_output,
         widgets.HBox([generate_button, export_button], layout=widgets.Layout(justify_content='center', margin='20px 0 0 0'))]
    )

    with plots_output:
        clear_output(wait=True)
        plt.show()

    tabs.selected_index = 2  # Navigate to the "Analytics" tab

In [ ]:
def generate_graph(button):
    global generated_figures
    generated_figures.clear()  # Clear previous figures
    # clear outputs
    with plots_output:
        clear_output(wait=True)

    # Get current selections from UI elements
    selected_users = [checkbox.description for checkbox in user_checkboxes if checkbox.value]
    selected_projects = [checkbox.description for checkbox in project_checkboxes if checkbox.value]
    graph_type = graph_type_dropdown.value

    # Validate selections and perform graph generation
    if selected_users and selected_projects:

        #**************** Contribution Breakdown ******************
        if graph_type == 'Contribution Breakdown':
            contribution_breakdown(selected_users, selected_projects)

        #**************** Activity Timeline ******************
        elif graph_type == 'Activity Timeline':
            activity_timeline(selected_users, selected_projects)

        #**************** Task Distribution ******************
        elif graph_type == 'Task Distribution':
            task_distribution(selected_users, selected_projects)

        #**************** user activity by tab ******************
        elif graph_type == 'User Activity by Tab':
            user_activity_by_tab(selected_users, selected_projects)
        #****************  user_productivity_heatmap ******************
        elif graph_type == 'User Productivity Heatmap':
            user_productivity_heatmap(selected_users, selected_projects)
        #**************** Weekend vs Weekday Work ******************
        elif graph_type == 'Weekend vs Weekday Work':
          weekend_vs_weekday_work(selected_users, selected_projects)
        #*********************** All Graphs *********************
        elif graph_type == 'All':
            all_graphs(selected_users, selected_projects)

        else:
            print("Invalid Graph Type selected.")

    else:
        print("Please select at least one user and one project.")







## Graphs

In [ ]:
def contribution_breakdown(selected_users, selected_projects):
    global generated_figures
    activity = {}
    for item in data:
        user = item.get("User")
        project = item.get("Document")
        if user in selected_users and project in selected_projects:
            month = int(item.get("Time").split('-')[1])
            activity.setdefault(user, [0]*12)[month - 1] += 1 #Remove?


    labels = list(activity.keys())
    sizes = np.sum(list(activity.values()), axis=1)

    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
    ax.set_title("Contribution Breakdown")
    generated_figures.append(fig)

    with plots_output:
        plt.show()

In [ ]:
def activity_timeline(selected_users, selected_projects):
    global generated_figures
    activity = {user: [0]*12 for user in selected_users}
    for item in data:
        user = item.get("User")
        project = item.get("Document")
        if user in selected_users and project in selected_projects:
            month = int(item.get("Time").split('-')[1])
            activity[user][month - 1] += 1

    fig, ax = plt.subplots()
    for user, activity_data in activity.items():
        ax.plot(activity_data, label=user)
    ax.set_xticks(range(12))
    ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    ax.set_title("Activity Timeline")
    ax.legend()
    generated_figures.append(fig)

    with plots_output:
        plt.show()

In [ ]:

def create_task_dataframe(selected_users, selected_projects):
    tasks = {}
    for item in data:
        user = item.get("User")
        project = item.get("Document")
        if user in selected_users and project in selected_projects:
            task_type = item.get("Description").split(' ')[0]
            tasks.setdefault(user, {}).setdefault(task_type, 0)
            tasks[user][task_type] += 1

    # Convert tasks dictionary to a DataFrame
    df = pd.DataFrame(tasks).fillna(0).astype(int)

    # Sort each student's column individually by the number of actions
    df = df.apply(lambda x: x.sort_values(ascending=False).values)

    # Calculate the sum of all actions and add it as a new row
    total_actions = df.sum(axis=0)
    df.loc['Total Actions'] = total_actions

    # Rename the index to "Actions"
    df.index.name = 'Actions'

    return df


In [ ]:
def task_distribution(selected_users, selected_projects):
    global generated_figures
    df = create_task_dataframe(selected_users, selected_projects)

    # Create a figure from the DataFrame
    fig, ax = plt.subplots(figsize=(12, 6))
    df.plot(kind='bar', ax=ax)
    ax.set_title("Task Distribution")
    ax.legend(title="Users")
    plt.tight_layout()
    generated_figures.append(fig)

    with plots_output:
        display(df)
        plt.show()


In [ ]:
def user_activity_by_tab(selected_users, selected_projects):
    global generated_figures
    activity = {user: {} for user in selected_users}
    for item in data:
        user = item.get("User")
        project = item.get("Document")
        tab = item.get("Tab")
        if user in selected_users and project in selected_projects:
            activity[user][tab] = activity[user].get(tab, 0) + 1

    fig, ax = plt.subplots(figsize=(12, 6))
    bottom = np.zeros(len(selected_users))
    for tab in set(tab for user_data in activity.values() for tab in user_data):
        values = [activity[user].get(tab, 0) for user in selected_users]
        ax.bar(selected_users, values, label=tab, bottom=bottom)
        bottom += values

    ax.set_xlabel('Users')
    ax.set_ylabel('Number of Actions')
    ax.set_title("User Activity by Tab")
    ax.legend(title="Tabs")
    plt.xticks(rotation=45, ha='right')
    generated_figures.append(fig)

    with plots_output:
        plt.show()

In [ ]:
def user_productivity_heatmap(selected_users, selected_projects):
    global generated_figures
    activity = {user: [0]*24 for user in selected_users}
    for item in data:
        user = item.get("User")
        project = item.get("Document")
        if user in selected_users and project in selected_projects:
            hour = int(item.get("Time").split()[1].split(':')[0])
            activity[user][hour] += 1

    df = pd.DataFrame(activity).T

    fig, ax = plt.subplots(figsize=(12, 6))
    sns.heatmap(df, cmap="YlOrRd", ax=ax)
    ax.set_xlabel('Hour of Day')
    ax.set_ylabel('Users')
    ax.set_title("User Productivity Heatmap")
    generated_figures.append(fig)

    with plots_output:
        plt.show()

In [ ]:
def weekend_vs_weekday_work(selected_users, selected_projects):
    global generated_figures

    work_distribution = {user: {'weekday': 0, 'weekend': 0} for user in selected_users}

    for item in data:
        user = item.get("User")
        project = item.get("Document")
        if user in selected_users and project in selected_projects:
            date_str = item.get("Time").split()[0]
            date_obj = datetime.strptime(date_str, '%Y-%m-%d')
            if date_obj.weekday() < 5:  # Monday = 0, Sunday = 6
                work_distribution[user]['weekday'] += 1
            else:
                work_distribution[user]['weekend'] += 1

    users = list(work_distribution.keys())
    weekday_work = [work_distribution[user]['weekday'] for user in users]
    weekend_work = [work_distribution[user]['weekend'] for user in users]

    fig, ax = plt.subplots(figsize=(12, 6))
    x = range(len(users))
    width = 0.35

    ax.bar([i - width/2 for i in x], weekday_work, width, label='Weekday')
    ax.bar([i + width/2 for i in x], weekend_work, width, label='Weekend')

    ax.set_ylabel('Number of Actions')
    ax.set_title('Weekday vs Weekend Work Distribution')
    ax.set_xticks(x)
    ax.set_xticklabels(users, rotation=45, ha='right')
    ax.legend()

    plt.tight_layout()
    generated_figures.append(fig)

    with plots_output:
        plt.show()

In [ ]:
def all_graphs(selected_users, selected_projects):
    contribution_breakdown(selected_users, selected_projects)
    activity_timeline(selected_users, selected_projects)
    task_distribution(selected_users, selected_projects)
    user_activity_by_tab(selected_users, selected_projects)
    user_productivity_heatmap(selected_users, selected_projects)
    weekend_vs_weekday_work(selected_users, selected_projects)

In [ ]:
def download_graphs(button):
    global generated_figures

    if not generated_figures:
        print("No graphs were generated. Please generate graphs before attempting to download.")
        return

    # Create a BytesIO object to store the PDF
    buffer = BytesIO()

    # Create the PDF document
    doc = SimpleDocTemplate(buffer, pagesize=letter)
    elements = []

    # Iterate through all the generated figures
    for fig in generated_figures:
        # Save the figure to a BytesIO object
        img_buffer = BytesIO()
        fig.savefig(img_buffer, format='png', dpi=300, bbox_inches='tight')
        img_buffer.seek(0)

        # Create an Image object and add it to the elements list
        img = Image(img_buffer)
        img.drawHeight = 6*inch
        img.drawWidth = 8*inch
        elements.append(img)

    # Build the PDF
    doc.build(elements)

    # Move the buffer's cursor to the beginning
    buffer.seek(0)

    # Generate a filename
    filename = f"onshape_analytics_{len(generated_figures)}_graphs.pdf"

    # Write the PDF content to a file
    with open(filename, 'wb') as f:
        f.write(buffer.getvalue())

    # Use google.colab.files to initiate the download
    files.download(filename)

    print(f"PDF file '{filename}' has been created and downloaded.")

    # Clear the generated figures list
    generated_figures.clear()

# Program

## Chatbot

In [ ]:
class OnShapeAnalyticsBot:
    def __init__(self, pairs, reflections, data):
        self._pairs = pairs
        self._reflections = reflections
        self.data = data

    def _wildcards(self, response, match):
        pos = response.find('%')
        while pos >= 0:
            num = int(response[pos+1:pos+2])
            response = response[:pos] + match.group(num) + response[pos+2:]
            pos = response.find('%')
        return response

    def respond(self, message):
        for pattern, responses in self._pairs:
            match = re.match(pattern, message.rstrip(".!"))
            if match:
                response = random.choice(responses)
                if callable(response):
                    return response()
                else:
                    return self._wildcards(response, match)

        if "how many" in message.lower() and "projects" in message.lower():
            return f"There are {len(data_documents)} projects in the database."
        elif "how many" in message.lower() and "users" in message.lower():
            return f"There are {len(set(item.get('User') for item in self.data))} users in the database."
        elif "how many" in message.lower() and "actions" in message.lower():
            return f"There are {len(self.data)} total actions in the database."
        elif "list all documents" in message.lower():
            return list_all_documents()
        elif "team member summary" in message.lower():
            return team_member_summary()
        elif "project activity overview" in message.lower():
            return project_activity_overview()
        elif "user activity ranking" in message.lower():
            return user_activity_ranking()
        elif "recent activity log" in message.lower():
            return recent_activity_log()

        return "I'm not sure how to respond to that. Can you please rephrase your question?"


def get_database_info():
    project_count = len(data_documents)
    user_count = len(set(item.get("User") for item in data))
    action_count = len(data)

    return f"There are {project_count} projects, {user_count} users, and {action_count} total actions in the database."

def list_all_documents():
    documents = get_documents_set(data)
    return "All Documents in the Database:\n" + "\n".join(f"{idx}. {doc}" for idx, doc in enumerate(documents, 1))

def team_member_summary():
    member_projects = {}
    for item in data:
        user = item.get("User")
        document = item.get("Document")
        member_projects.setdefault(user, set()).add(document)

    summary = "Team Member Project Summary:\n"
    for member, projects in member_projects.items():
        summary += f"{member}: {len(projects)} projects\n"
        for project in projects:
            summary += f"  - {project}\n"
        summary += "\n"
    return summary

def project_activity_overview():
    project_actions = {}
    for item in data:
        document = item.get("Document")
        project_actions[document] = project_actions.get(document, 0) + 1

    overview = "Project Activity Overview:\n"
    for project, actions in sorted(project_actions.items(), key=lambda x: x[1], reverse=True):
        overview += f"{project}: {actions} actions\n"
    return overview

def user_activity_ranking():
    user_actions = {}
    for item in data:
        user = item.get("User")
        user_actions[user] = user_actions.get(user, 0) + 1

    ranking = "User Activity Ranking:\n"
    for rank, (user, actions) in enumerate(sorted(user_actions.items(), key=lambda x: x[1], reverse=True), 1):
        ranking += f"{rank}. {user}: {actions} actions\n"
    return ranking

def recent_activity_log(limit=10):
    sorted_data = sorted(data, key=lambda x: x.get("Time"), reverse=True)

    log = f"Recent Activity Log (Last {limit} actions):\n"
    for item in sorted_data[:limit]:
        log += f"{item['Time']} - {item['User']} - {item['Document']} - {item['Description']}\n"
    return log
# Define the patterns and reflections outside to be passed into the bot
patterns = [
    (r'hi|hello|hey', ['Hello!', 'Hi there!', 'Hey!']),
    (r'how are you?', ['I\'m good, thank you!', 'I\'m doing well, thanks for asking.']),
    (r'what is your name?', ['You can call me OnShapeAnalyticsBot.', 'I go by the name OnShapeAnalyticsBot.']),
    (r'my name is (.*)', ['Nice to meet you, %1.', 'Hello, %1!']),
    (r'thank you (.*)', ['You\'re welcome!', 'Happy to help!']),
    (r'What is the purpose of the system ?', ['The purpose of the system is to help software managers improve the team\'s performance by analyzing their work.']),
    (r'what\'s included in the software', ['The software includes an action counter, graphs for every action per worker/student, and a notification page for the manager to see important user activity or system notifications.']),
    (r'what is onshape|onshape|on shape', ['Onshape is a cloud-based CAD platform for product design and development. Learn more here: https://www.onshape.com/']),
    (r'how to filter the graph', ['You can filter the graph by clicking on the desirable data.']),
    (r'how to add a new team member', ['At this moment of development, we don\'t have this feature yet.']),
    (r'how to show important actions', ['Important actions are individual. You can filter them to your liking.']),
    (r'how to show who did the most actions', ['If you go to the team graph section, you can filter it with actions and see who contributed the most.']),
    (r'tell me about the database', [get_database_info]),
    (r'list all documents', [list_all_documents]),
    (r'team member summary', [team_member_summary]),
    (r'project activity overview', [project_activity_overview]),
    (r'user activity ranking', [user_activity_ranking]),
    (r'recent activity log', [recent_activity_log]),
    (r'quit|exit|bye', ['Goodbye!', 'See you later!', 'Bye!']),
]
# Global variable to store chatbot creation status
chatbot_created = False
chatbot_box = None

def run_chatbot():
    global chatbot_created, chatbot_box

    if chatbot_created:
        chatbot_box.layout.display = 'inline-block'
        return

    chatbot = OnShapeAnalyticsBot(patterns, {}, data)

    # Chatbot UI components
    input_box = widgets.Text(
        placeholder='Type your message here',
        layout=widgets.Layout(height='100px', width='400px', font_size='20px', padding='10px', box_sizing='border-box', margin='30px')
    )
    send_button = widgets.Button(
        description="Send",
        layout=widgets.Layout(height='50px', width='120px', font_size='20px', margin='30px')
    )
    output = widgets.Output(
        layout=widgets.Layout(overflow='auto', border='2px solid black', height='300px', width='520px')
    )

    # "X" button to close/hide the chatbot
    close_button = widgets.Button(
        description="X",
        layout=widgets.Layout(width='30px', height='30px', padding='0px', font_size='20px')
    )

    # Wrap the close_button in an HBox and align it to the right
    close_button_container = widgets.HBox([close_button], layout=widgets.Layout(justify_content='flex-end'))

    chat_layout = widgets.VBox([widgets.HBox([input_box, send_button]), output])
    chatbot_box = widgets.VBox([close_button_container, chat_layout])

    # Initially hide the chatbot
    chatbot_box.layout.display = 'inline-block'

    chatbot_created = True

    def on_send_button_clicked(b):
        user_input = input_box.value
        input_box.value = ''

        with output:
            print(f"You: {user_input}")
            response = chatbot.respond(user_input)
            print(f"OnShapeAnalyticsBot: {response}")

        if user_input.lower() in ['quit', 'exit', 'bye']:
            send_button.disabled = True
            input_box.disabled = True

    def on_close_button_clicked(b):
        chatbot_box.layout.display = 'none'
        chatbot_button.layout.display = 'inline-block'

    send_button.on_click(on_send_button_clicked)
    close_button.on_click(on_close_button_clicked)

    display(chatbot_box)

    with output:
        print("Hello! I'm OnShapeAnalyticsBot. How can I help you today?")

def on_chatbot_button_clicked(b):
    chatbot_button.layout.display = 'inline-block'

def on_chatbot_button_clicked(b):
    run_chatbot()

def on_chatbot_button_clicked(b):
    run_chatbot()
    chatbot_button.layout.display = 'none'





## Initialisation

In [ ]:
data = get_data_from_firebase('https://test100000-ccdb4-default-rtdb.asia-southeast1.firebasedatabase.app/', '/team_data2', 'data')
data_documents = get_documents_set(data)

## Widgets

In [ ]:
style = widgets.HTML("""
    <style>
      html, body {
        height: 100vh;
        margin: 0; /* Remove default margin */
        padding: 0; /* Remove default padding */
      }
      body {
        font-family: 'Times New Roman', serif;
        background: rgb(8,129,113);
        background: linear-gradient(90deg, rgba(8,129,113,1) 0%, rgba(237,237,237,1) 67%);
        border: none; /* Remove any border */
      }
      ul {
        padding-left: 20px;
        margin-top: 10px;
        margin-bottom: 10px;
        list-style-type: disc;
        justify-content: space-between;
      }
      .jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab {
        flex: 1 1 var(--jp-widgets-horizontal-tab-width);
        font-weight: bold;
        background-color: transparent;
        color: black;
        border: 1px solid black;
        border-radius: 5px;
        padding: 10px;
        transition: background-color 0.3s, border 0.3s;
      }
      .jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab:hover {
         background-color: black;
         color: white;
         border: 1px solid black;
         cursor: pointer;
      }
      .jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab.p-mod-current {
        background-color: black;
        color: white;
      }
      li {
        font-size: 20px !important;
        font-weight: bold;
        margin-bottom: 5px;
        padding: 10px;
        gap: 10px;
        background-color: #ffffff; /* White background for list items */
        border: solid 1px #dcdcdc; /* Light gray border for list items */
        border-radius: 5px; /* Rounded corners for list items */
      }
      h3 {
        text-decoration: underline;
      }
      h1 {
        text-decoration: underline;
        padding: 20px;
        color: black;
      }
      span {
        font-size: 20px !important;
      }
      select, option {
        font-family: 'Times New Roman', serif;
        font-size: 20px !important;
      }
      .widget-label {
        font-size: 22px;
      }
      .widget-button {
        background-color: #007bff;
        color: white;
        height: 50px;
        font-size: 24px;
        font-weight: bold;
        border: none;
        border-radius: 5px;
        padding: 10px;
        cursor: pointer;
      }
      .widget-button:hover {
        background-color: #0056b3;
      }
      .lm-Widget.p-Widget.jupyter-widgets.jupyter-button.widget-button.mod-info {
        margin-top: 100px;
        margin-left: 20px;
        width: 300px;
      }
      .lm-Widget.p-Widget.lm-Panel.p-Panel.p-TabPanel-tabContents.widget-tab-contents {
        background: rgb(8,129,113);
        background: linear-gradient(90deg, rgba(8,129,113,1) 0%, rgba(237,237,237,1) 67%);
        flex-grow: 1;
        border: none;
      }
      .black-text-output .jupyter-widgets-output-area pre {
        color: black !important;
      }
      .output_subarea.output_text pre {
      color: black !important;
      font-size: 20px;
      padding: 10px;
      margin: 5px;
    }
    input {
        font-size: 20px !important;
        font-weight: bold;
    }
    </style>
""")




In [ ]:
generated_figures = []
## projects tab

# header
project_header = widgets.HTML("<h2>Select Projects</h2>")

# checkboxs
project_checkboxes = [
    widgets.Checkbox(
        value=False,
        description=str(doc),
    ) for doc in data_documents if doc is not None
]

# select button
select_projects_button = widgets.Button(
    description="Next",
)

select_projects_button.on_click(update_members)

# layout
projects_tab = widgets.VBox([
        project_header,
        widgets.VBox(project_checkboxes),
        widgets.HBox([select_projects_button], layout=widgets.Layout(justify_content='center', margin='20px 0 0 0'))
    ])

In [ ]:
## members tab

user_checkboxes = []

# start empty layout
members_tab = widgets.VBox([widgets.Label("Select projects to continue")])

# header
members_header = widgets.HTML("<h2>Select Team Members</h2>")

# select button
select_members_button = widgets.Button(
    description="Next",
)
select_members_button.on_click(update_analytics)

# full layout in function update_members()

In [ ]:
## analytics tab

plots_output = widgets.Output()


# Dropdown menu
graph_type_dropdown = widgets.Dropdown(
    options=['Contribution Breakdown', 'Activity Timeline', 'Task Distribution', 'User Activity by Tab', 'User Productivity Heatmap', 'Weekend vs Weekday Work', 'All'],
    description='Select Graph Type:',
    style={'description_width': 'initial', 'description_font_size': '40px'},
    layout={'width': '450px'}
)


# generate button
generate_button = widgets.Button(
    description="Generate",
)
# generate button
export_button = widgets.Button(
    description="Export",
)

# Spacer
spacer = widgets.Box(layout=widgets.Layout(width='20px'))
export_button.on_click(download_graphs)
generate_button.on_click(generate_graph)

# start empty layout
analytics_content = widgets.VBox([widgets.Label("Select projects and members to continue")])

# full layout in function update_analytics()

## Main

In [ ]:
# header
header = widgets.HTML("<h1>OnShape Analytics Program</h1>")

# tabs
tabs = widgets.Tab()
tabs.children = [
    projects_tab,
    members_tab,
    analytics_content
]
tabs.set_title(0, 'Select Projects')
tabs.set_title(1, 'Select Members')
tabs.set_title(2, 'Analytics')

# chatbot button
chatbot_button = widgets.Button(
    description="Chat with Bot",
    button_style='info'
)
chatbot_box = widgets.VBox([chatbot_button])

# layout and display
app_layout = widgets.VBox([style, header, tabs, chatbot_box])


chatbot_button.on_click(on_chatbot_button_clicked)

display(app_layout)